# Week3_1 Assignment

## [BASIC](#Basic) 
- 토크나이징이 완료된 위키 백과 코퍼스를 다운받고 **단어 사전을 구축하는 함수를 구현**할 수 있다.
- `Skip-Gram` 방식의 학습 데이터 셋을 생성하는 **Dataset과 Dataloader 클래스를 구현**할 수 있다.
- **Negative Sampling** 함수를 구현할 수 있다. 


## [CHALLENGE](#Challenge)
- Skip-Gram을 학습 과정 튜토리얼을 따라하며, **Skip-Gram을 학습하는 클래스를 구현**할 수 있다. 


## [ADVANCED](#Advanced)
- Skip-Gram 방식으로 word embedding을 학습하는 **Word2Vec 클래스를 구현**하고 실제로 학습할 수 있다.
- 학습이 완료된 word embedding을 불러와 **Gensim 패키지를 사용해 유사한 단어**를 뽑을 수 있다. 

### Reference
- [Skip-Gram negative sampling 한국어 튜토리얼](https://wikidocs.net/69141)
    - (참고) 위 튜토리얼에서는 target word와 context word 페어의 레이블은 1로, target word와 negative sample word 페어의 레이블은 0이 되도록 학습 데이터를 구현해 binary classification을 구현한다. 하지만 우리는 word2vec 논문 방식을 그대로 따르기 위해 label을 생성하지 않고 대신 loss 함수를 변행해서 binary classification을 학습할 것이다. 

In [1]:
import os 
import sys
import pandas as pd
import numpy as np
import re
from typing import List, Dict
import random

In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 8.8 MB/s 
     |████████████████████████████████| 895 kB 50.4 MB/s 
     |████████████████████████████████| 6.5 MB 50.3 MB/s 
     |████████████████████████████████| 596 kB 54.5 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [4]:
# seed
seed = 7777
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [5]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla T4
cuda


## Basic

### 토크나이징이 완료된 위키 백과 코퍼스 다운로드 및 불용어 사전 크롤링
- 나의 구글 드라이브에 데이터를 다운받아 영구적으로 사용할 수 있도록 하자. 
    - [데이터 다운로드 출처](https://ratsgo.github.io/embedding/downloaddata.html)
- 다운받은 데이터는 토크나이징이 완료된 상태이지만 불용어를 포함하고 있다. 따라서 향후 불용어를 제거하기 위해 불용어 사전을 크롤링하자. 
    - [불용어 사전 출처](https://www.ranks.nl/stopwords/korean)

In [6]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
cd  ./drive/MyDrive/NLP

/content/drive/MyDrive/NLP


In [8]:
# # 데이터 다운로드
# !pip install gdown
# !gdown https://drive.google.com/u/0/uc?id=1Ybp_DmzNEpsBrUKZ1-NoPDzCMO39f-fx
# !unzip tokenized.zip

In [9]:
# 한국어 불용어 리스트 크롤링
import requests
from bs4 import BeautifulSoup

url = "https://www.ranks.nl/stopwords/korean"
response = requests.get(url, verify = False)

if response.status_code == 200:
    soup = BeautifulSoup(response.text,'html.parser')
    content = soup.select_one('#article178ebefbfb1b165454ec9f168f545239 > div.panel-body > table > tbody > tr')
    stop_words=[]
    for x in content.strings:
        x=x.strip()
        if x:
            stop_words.append(x)
    print(f"# Korean stop words: {len(stop_words)}")
else:
    print(response.status_code)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


# Korean stop words: 677


In [10]:
stop_words[0]

'아'

### 단어 사전 구축 함수 구현 
- 문서 리스트를 입력 받아 사전을 생성하는 `make_vocab()` 함수를 구현하라.
- 함수 정의
    - 입력 매개변수
        - docs : 문서 리스트
        - min_count : 최소 단어 등장 빈도수 (단어 빈도가 `min_count` 미만인 단어는 사전에 포함하지 않음)
    - 조건
        - 문서 길이 제한
            - 단어 개수가 3개 이하인 문서는 처리하지 않음. (skip)
        - 사전에 포함되는 단어 빈도수 제한
            - 단어가 빈도가 `min_count` 미만은 단어는 사전에 포함하지 않음.
        - 불용어 제거 
            - 불용어 리스트에 포함된 단어는 제거 
    - 반환값 
        - word2count : 단어별 빈도 사전 (key: 단어, value: 등장 횟수)
        - wid2word : 단어별 인덱스(wid) 사전 (key: 단어 인덱스(int), value: 단어)
        - word2wid : 인덱스(wid)별 단어 사전 (key: 단어, value: 단어 인덱스(int))

In [11]:
# 코퍼스 로드
with open("/content/drive/MyDrive/NLP/tokenized/wiki_ko_mecab.txt",'r') as file:
  docs = file.readlines() 

In [12]:
print(f"# wiki documents: {len(docs):,}")

# wiki documents: 311,237


In [13]:
# 문서 개수를 500개로 줄임
docs=random.sample(docs,500)

In [14]:
print(f"# wiki documents: {len(docs):,}")

# wiki documents: 500


In [15]:
# 문서 내 숫자, 영어 대소문자, 특수문자를 제거 (re package 사용)
import re

new_docs = []
for doc in docs:
  doc = re.sub(r"[^ ㄱ-ㅣ가-힣+]","",doc)
  new_docs.append(doc)

docs = new_docs

In [16]:
print(f"Check : {docs[488][:1000]}")

Check : 히 오 돈목    은 현존 하 는  종 의 어류 로 이루어진 비교 적 원시 적 인 조기어류 목 의 하나 이 다  이 들 은 전통 적 으로 골설어 목 으로 분류 해 왔으며  많 은 학자 들 은 아직 도 그렇 게 분류 하 지만  멸종 한 얀 비 아니 아 속    에 관한 화석 연구 는 히 오 돈목 이 다른 골설어 목 어류 와 초기 에 분리 되 었 음 을 시사 하 고 있 으며  이 때문 에 최근 에 는 별도 의 목 으로 분류 하 고 있 다  생명 의 나무      프로젝트 의 히 오 돈목    정보


In [17]:
def make_vocab(docs:List[str], min_count:int):
    """
    'docs'문서 리스트를 입력 받아 단어 사전을 생성.
    
    return 
        - word2count : 단어별 빈도 사전
        - wid2word : 단어별 인덱스(wid) 사전 
        - word2wid : 인덱스(wid)별 단어 사전
    """

    word2count = dict()
    word2id = dict()
    id2word = dict()

    _word2count={}
    for doc in tqdm(docs):
        word_list = doc.split()
        # 1. 문서 길이 제한 -> 단어개수 3개 이하
        # 2. 임시 딕셔너리(_word2count)에 단어별 등장 빈도 기록
        # 3. 불용어 제거
        if len(word_list)<4 : continue

        for word in word_list:
          if word not in _word2count:_word2count[word] = 1
          else: _word2count[word] +=1
    
    for word in _word2count:
      if word in stop_words: continue
    # 4. 토큰 최소 빈도를 만족하는 토큰만 사전에 추가      
      if _word2count[word] < min_count:continue
      else: word2count[word] = _word2count[word]

    id=0
    for word in word2count:
      word2id[word] = id
      id2word[id] = word
      id+=1

    return word2count, word2id, id2word

In [18]:
word2count, word2id, id2word = make_vocab(docs, min_count=5)

100%|██████████| 500/500 [00:00<00:00, 6888.44it/s]


In [19]:
doc_len = sum(word2count.values()) # 문서 내 모든 단어의 개수 (단어별 등장 빈도의 총 합)
print(f"{doc_len:,}")

161,322


In [20]:
print(f"# unique word : {len(word2id):,}")

# unique word : 5,662


### Dataset 클래스 구현
- Skip-Gram 방식의 학습 데이터 셋(`Tuple(target_word, context_word)`)을 생성하는 `CustomDataset` 클래스를 구현하라.
- 클래스 정의
    - 생성자(`__init__()` 함수) 입력 매개변수
        - docs: 문서 리스트
        - word2id: 단어별 인덱스(wid) 사전
        - window_size: Skip-Gram의 윈도우 사이즈
    - 메소드
        - `make_pair()`
            - 문서를 단어로 쪼개고, 사전에 존재하는 단어들만 단어 인덱스로 변경
            - Skip-gram 방식의 `(target_word, context_word)` 페어(tuple)들을 `pairs` 리스트에 담아 반환
        - `__len__()`
            - `pairs` 리스트의 개수 반환
        - `__getitem__(index)`
            - `pairs` 리스트를 인덱싱
    - 주의 사항
        - `nn.Module`를 부모 클래스로 상속 받음 


In [21]:
class CustomDataset(Dataset):
    """
    문서 리스트를 받아 skip-gram 방식의 (target_word, context_word) 데이터 셋을 생성
    """
    def __init__(self, docs:List[str], word2id:Dict[str,int], window_size:int=5):
        self.docs = docs
        self.word2id = word2id
        self.window_size = window_size
        self.pairs = self.make_pair()
    
    def make_pair(self):
        """
        (target, context) 형식의 Skip-gram pair 데이터 셋 생성 
        """
        pairs = []
        for doc in self.docs:
          word_list = doc.split()
          for i, word in enumerate(word_list):
            if word in self.word2id : word_list[i] = self.word2id[word]

          for i in range(len(word_list)):
            if type(word_list[i]) != type(1):continue
            min_index = max(0,i-self.window_size)
            max_index = min(len(word_list)-1,i+self.window_size)

            for j in range(min_index,max_index+1):
              if type(word_list[j]) != type(1) : continue
              if i == j : continue

              pairs.append((word_list[i],word_list[j]))
        
        return pairs
        
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        return self.pairs[idx]

In [22]:
dataset = CustomDataset(docs, word2id, window_size=5)

In [23]:
len(dataset)

1024780

In [24]:
# verify (target word, context word)
for i, pair in enumerate(dataset):
    if i==100:
        break
    print(f"({id2word[pair[0]]}, {id2word[pair[1]]})")
    

(남모, 공주)
(남모, 는)
(남모, 신라)
(남모, 공주)
(공주, 남모)
(공주, 는)
(공주, 신라)
(공주, 공주)
(공주, 왕족)
(는, 남모)
(는, 공주)
(는, 신라)
(는, 공주)
(는, 왕족)
(신라, 남모)
(신라, 공주)
(신라, 는)
(신라, 공주)
(신라, 왕족)
(공주, 남모)
(공주, 공주)
(공주, 는)
(공주, 신라)
(공주, 왕족)
(왕족, 공주)
(왕족, 는)
(왕족, 신라)
(왕족, 공주)
(왕족, 공주)
(공주, 왕족)
(공주, 부여)
(공주, 씨)
(공주, 딸)
(부여, 공주)
(부여, 씨)
(부여, 딸)
(부여, 며)
(씨, 공주)
(씨, 부여)
(씨, 딸)
(씨, 며)
(씨, 백제)
(딸, 공주)
(딸, 부여)
(딸, 씨)
(딸, 며)
(딸, 백제)
(며, 부여)
(며, 씨)
(며, 딸)
(며, 백제)
(며, 였)
(백제, 씨)
(백제, 딸)
(백제, 며)
(백제, 였)
(백제, 다)
(였, 며)
(였, 백제)
(였, 다)
(였, 인)
(였, 준정)
(다, 백제)
(다, 였)
(다, 인)
(다, 준정)
(인, 였)
(인, 다)
(인, 준정)
(인, 신라)
(준정, 였)
(준정, 다)
(준정, 인)
(준정, 신라)
(준정, 초대)
(신라, 인)
(신라, 준정)
(신라, 초대)
(신라, 여성)
(신라, 화랑)
(초대, 준정)
(초대, 신라)
(초대, 여성)
(초대, 화랑)
(초대, 였)
(초대, 다)
(여성, 신라)
(여성, 초대)
(여성, 화랑)
(여성, 였)
(여성, 다)
(화랑, 신라)
(화랑, 초대)
(화랑, 여성)
(화랑, 였)
(화랑, 다)
(화랑, 준정)
(였, 초대)
(였, 여성)
(였, 화랑)


### 위에서 생성한 `dataset`으로 DataLoader  객체 생성
- `DataLoader` 클래스로 `train_dataloader`객체를 생성하라. 
    - 생성자 매개변수와 값
        - dataset = 위에서 생성한 dataset
        - batch_size = 64
        - shuffle = True

In [25]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset, batch_size = 64, shuffle=True)

In [26]:
len(train_dataloader)

16013

### Negative Sampling 함수 구현
- Skip-Gram은 복잡도를 줄이기 위한 방법으로 negative sampling을 사용한다. 
- `sample_table`이 다음과 같이 주어졌을 때, sample_table에서 랜덤으로 값을 뽑아 (batch_size, n_neg_sample) shape의 matrix를 반환하는 `get_neg_v_negative_sampling()`함수를 구현하라. 
- Sample Table은 negative distribution을 따른다. 
    - [negative distribution 설명](https://aegis4048.github.io/optimize_computational_efficiency_of_skip-gram_with_negative_sampling#How-are-negative-samples-drawn?)
- 함수 정의
    - 입력 매개변수
        - batch_size : 배치 사이즈, matrix의 row 개수 
        - n_neg_sample : negative sample의 개수, matrix의 column 개수
    - 반환값 
        - neg_v : 추출된 negative sample (2차원의 리스트)


In [27]:
# negative sample을 추출할 sample table 생성 (해당 코드를 참고)
sample_table = []
sample_table_size = doc_len

# noise distribution 생성
alpha = 3/4
frequency_list = np.array(list(word2count.values())) ** alpha
Z = sum(frequency_list)
ratio = frequency_list/Z
negative_sample_dist = np.round(ratio*sample_table_size)

for wid, c in enumerate(negative_sample_dist):
    sample_table.extend([wid]*int(c))

In [28]:
len(sample_table)

161159

In [29]:
print(word2count.values())
print(negative_sample_dist)

dict_values([6, 52, 5895, 27, 15, 20, 189, 70, 486, 8, 789, 5722, 1002, 9, 52, 88, 160, 5, 10, 16, 3711, 30, 30, 686, 1744, 1589, 9, 11, 318, 11, 62, 404, 77, 250, 5, 11, 207, 63, 10, 373, 17, 46, 46, 62, 21, 64, 200, 769, 131, 27, 979, 1120, 1951, 5, 2364, 38, 158, 364, 271, 123, 136, 13, 66, 17, 14, 10, 31, 107, 9, 94, 21, 139, 55, 10, 51, 10, 244, 5, 6, 473, 10, 5, 94, 2488, 74, 563, 370, 292, 8, 21, 477, 692, 2384, 39, 158, 314, 964, 10, 100, 36, 222, 98, 13, 18, 15, 13, 163, 47, 75, 5, 15, 7, 51, 8, 272, 108, 5, 173, 46, 41, 161, 156, 5, 15, 30, 21, 62, 193, 115, 63, 52, 8, 128, 180, 45, 17, 9, 47, 19, 1006, 5, 134, 73, 232, 676, 10, 10, 108, 131, 9, 118, 7, 16, 8, 28, 15, 112, 22, 22, 5, 43, 25, 107, 191, 5, 14, 8, 57, 75, 54, 6, 54, 9, 5, 20, 21, 35, 15, 7, 57, 55, 26, 13, 19, 5, 9, 30, 5, 11, 138, 259, 26, 53, 11, 59, 6, 9, 8, 12, 10, 17, 7, 51, 313, 13, 9, 15, 8, 22, 28, 9, 21, 108, 6, 97, 26, 17, 103, 6, 54, 38, 104, 702, 6, 14, 6, 21, 12, 9, 6, 26, 9, 61, 7, 74, 11, 16, 16, 

In [30]:
def get_neg_v_negative_sampling(batch_size:int, n_neg_sample:int):
    """
    위에서 정의한 sample_table에서 (batch_size, n_neg_sample) shape만큼 랜덤 추출해 "네거티브 샘플 메트릭스"를 생성
    np.random.choice() 함수 활용 (위에서 정의한 sample_table을 함수의 argument로 사용)
    """
    
    neg_v = np.random.choice(sample_table,(batch_size,n_neg_sample)).tolist()
    
    return neg_v

In [31]:
get_neg_v_negative_sampling(4, 5)

[[36, 4419, 52, 2799, 2018],
 [5555, 307, 4185, 371, 5439],
 [3478, 4770, 1336, 271, 180],
 [57, 3457, 1687, 4851, 1633]]

## Challenge

### 미니 튜토리얼
- 아래 튜토리얼을 따라하며 Skip-Gram 모델의 `forward` 및 `loss` 연산 방식을 이해하자
- Reference
    - [torch.nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)
    - [torch bmm](https://pytorch.org/docs/stable/generated/torch.bmm.html)
    - [Skip-Gram negative sampling loss function 설명 영문 블로그](https://aegis4048.github.io/optimize_computational_efficiency_of_skip-gram_with_negative_sampling#Derivation-of-Cost-Function-in-Negative-Sampling)
    - [Skip-Gram negative sampling loss function 설명 한글 블로그](https://reniew.github.io/22/)

In [32]:
# hyper parameter example
emb_size = 30000 # vocab size
emb_dimension = 300 # word embedding 차원
n_neg_sample = 5
batch_size = 32

In [33]:
# 1. Embedding Matrix와 Context Matrix를 생성
u_embedding = nn.Embedding(emb_size, emb_dimension, sparse=True).to(device)
v_embedding = nn.Embedding(emb_size, emb_dimension, sparse=True).to(device)

In [34]:
# 2. wid(단어 인덱스)를 임의로 생성
pos_u = torch.randint(high = emb_size, size = (batch_size,))
pos_v = torch.randint(high = emb_size, size = (batch_size,))
neg_v = get_neg_v_negative_sampling(batch_size, n_neg_sample)
print(f"Target word idx : {pos_u} Pos context word idx : {pos_v} Neg context word idx : {neg_v}\n")

Target word idx : tensor([24460, 10634,  2864, 23952,  3320, 15187, 19625, 26546, 27339,  3920,
        25847,  6023,  5055,  7070,  6291, 10245, 15926,   641, 20178,  4565,
         4784, 26715, 16955, 28742, 17947, 19774,  8065, 22605,  3061, 28965,
         3056, 17963]) Pos context word idx : tensor([23224,  5636, 23712,  5234,  3991, 17897, 25123, 17938, 19634, 24228,
          693,   799, 25457,  1308, 28935, 25696,  5601, 23878,  8312,  1292,
        21380, 16974,  9318,  9578, 12915, 29271, 26465, 20572,  2362, 25929,
        19754, 29080]) Neg context word idx : [[870, 3538, 898, 92, 628], [1013, 4847, 5514, 4234, 1501], [4707, 625, 1568, 759, 1982], [520, 1871, 3981, 1536, 362], [56, 3895, 2548, 14, 5258], [1031, 549, 684, 4225, 623], [2914, 1328, 1041, 2770, 2578], [702, 3038, 4881, 93, 4957], [3875, 285, 115, 2513, 331], [1091, 2825, 538, 114, 3724], [1486, 753, 1631, 4721, 5034], [871, 1909, 3140, 465, 1338], [3282, 346, 1813, 470, 87], [621, 4216, 2544, 4083, 52], [1644, 

In [35]:
# 3. tensor로 변환
pos_u = Variable(torch.LongTensor(pos_u)).to(device)
pos_v = Variable(torch.LongTensor(pos_v)).to(device)
neg_v = Variable(torch.LongTensor(neg_v)).to(device)

In [36]:
# 4. wid로 각각의 embedding matrix에서 word embedding 값을 가져오기
pos_u = u_embedding(pos_u)
pos_v = v_embedding(pos_v)
neg_v = v_embedding(neg_v)
print(f"shape of pos_u embedding : {pos_u.shape}\n shape of pos_v embedding : {pos_v.shape}\n shape of neg_v embedding : {neg_v.shape}")


shape of pos_u embedding : torch.Size([32, 300])
 shape of pos_v embedding : torch.Size([32, 300])
 shape of neg_v embedding : torch.Size([32, 5, 300])


In [37]:
# 5. dot product 
pos_score = torch.mul(pos_u, pos_v) # 행렬 element-wise 곱
pos_score = torch.sum(pos_score, dim=1)
print(f"shape of pos logits : {pos_score.shape}\n")

neg_score = torch.bmm(neg_v, pos_u.unsqueeze(dim=2)).squeeze()
print(f"shape of logits : {neg_score.shape}")

shape of pos logits : torch.Size([32])

shape of logits : torch.Size([32, 5])


In [38]:
# 6. loss 구하기
pos_score = F.logsigmoid(pos_score)
neg_score = F.logsigmoid(-1*neg_score) # negative의 logit은 minimize 하기 위해 -1 곱함
print(f"pos logits : {pos_score.sum()}")
print(f"neg logits : {neg_score.sum()}")
loss = -1 * (torch.sum(pos_score) + torch.sum(neg_score))
print(f"Loss : {loss}")

pos logits : -241.4199676513672
neg logits : -1373.56982421875
Loss : 1614.98974609375


### Skip-gram 클래스 구현
- Skip-Gram 방식으로 단어 embedding을 학습하는 `SkipGram` 클래스를 구현하라.
- 클래스 정의
    - 생성자(`__init__()` 함수) 입력 매개변수
        - `vocab_size` : 사전내 단어 개수
        - `emb_dimension` : 엠베딩 크기
        - `device` : 연산 장치 종류
    - 생성자에서 생성해야할 변수 
        - `vocab_size` : 사전내 단어 개수
        - `emb_dimension` : 엠베딩 크기
        - `u_embedding` : (vocab_size, emb_dimension) 엠베딩 메트릭스 (target_word)
        - `v_embedding` : (vocab_size, emb_dimension) 엠베딩 메트릭스 (context_word)
    - 메소드
        - `init_embedding()` (제공됨)
            - 엠베딩 메트릭스 값을 초기화
        - `forward()`
            - 위 튜토리얼과 같이 dot product를 수행한 후 score를 생성
            - loss를 반환 (loss 설명 추가)
        - `save_emedding()` (제공됨)
            - `u_embedding`의 단어 엠베딩 값을 단어 별로 파일에 저장
    - 주의 사항     
        - `nn.Module`를 부모 클래스로 상속 받음 

In [45]:
class SkipGram(nn.Module):
    def __init__(self, vocab_size:int, emb_dimension:int, device:str):
        super(SkipGram, self).__init__()
        self.vocab_size = vocab_size
        self.emb_dimension = emb_dimension
        self.u_embedding = nn.Embedding(vocab_size, emb_dimension, sparse=True).to(device)
        self.v_embedding = nn.Embedding(vocab_size, emb_dimension, sparse=True).to(device)
        self.init_embedding()
    
    
    def init_embedding(self):
        """
        u_embedding과 v_embedding 메트릭스 값을 초기화
        """
        initrange = 0.5 / self.emb_dimension
        self.u_embedding.weight.data.uniform_(-initrange, initrange)
        self.v_embedding.weight.data.uniform_(-0, 0)
    
    
    def forward(self, pos_u, pos_v, neg_v):
        """
        dot product를 수행한 후 score를 생성
        loss 반환
        """    
            
        # 각각의 embedding matrix에서 word embedding 값을 가져오기
        pos_u = self.u_embedding(pos_u)
        pos_v = self.v_embedding(pos_v)
        neg_v = self.v_embedding(neg_v)

        # dot product 
        pos_score = torch.sum(torch.mul(pos_u, pos_v), dim=1)
        neg_score = torch.bmm(neg_v, pos_u.unsqueeze(dim=2)).squeeze()
        
        # loss 구하기
        pos_score = F.logsigmoid(pos_score)
        neg_score = F.logsigmoid(-1*neg_score) # negative의 logit은 minimize 하기 위해 -1 곱함
        loss = -1 * (torch.sum(pos_score) + torch.sum(neg_score))

        return loss
    
    def save_embedding(self, id2word, file_name, use_cuda):
        """
        'file_name' 위치에 word와 word_embedding을 line-by로 저장
        파일의 첫 줄은 '단어 개수' 그리고 '단어 embedding 사이즈' 값을 입력해야 함
        """
        if use_cuda: # parameter를 gpu 메모리에서 cpu 메모리로 옮김
            embedding = self.u_embedding.weight.cpu().data.numpy()
        else:
            embedding = self.u_embedding.weight.data.numpy()

        with open(file_name, 'w') as writer:
            # 파일의 첫 줄은 '단어 개수' 그리고 '단어 embedding 사이즈' 값을 입력해야 함
            writer.write(f"{len(id2word)} {embedding.shape[-1]}\n")
            
            for wid, word in id2word.items():
                e = embedding[wid]
                e = " ".join([str(e_) for e_ in e])
                writer.write(f"{word} {e}\n")

## Advanced

### Skip-Gram 방식의  Word2Vec 클래스 구현
- Skip-Gram 방식으로 단어 embedding을 학습하는 `Word2Vec` 클래스를 구현하라.
- 클래스 정의
    - 생성자(`__init__()`) 입력 매개 변수
        - `input_file` : 학습할 문서 리스트
        - `output_file_name` : 학습된 word embedding을 저장할 파일 위치
        - `device` : 연상 장치 종류
        - `emb_dimension` : word embedding 차원
        - `batch_size` : 학습 배치 사이즈
        - `window_size` : skip-gram 윈도우 사이즈 (context word 개수를 결정)
        - `n_neg_sample` : negative sample 개수
        - `iteration` : 학습 반복 횟수
        - `lr` : learning rate
        - `min_count` : 사전에 추가될 단어의 최소 등장 빈도
    - 생성자에서 생성해야 할 변수 
        - `docs` : 학습할 문서 리스트
        - `output_file_name` : 학습된 word embedding을 저장할 파일 위치
        - `word2count`, `word2id`, `id2word` : 위에서 구현한 `make_vocab()` 함수의 반환 값
        - `device` : 연산 장치 종류
        - `emb_size` : vocab의 (unique한) 단어 종류 
        - `emb_dimension` : word embedding 차원
        - `batch_size` : 학습 배치 사이즈
        - `window_size` : skip-gram 윈도우 사이즈 (context word 개수를 결정)
        - `n_neg_sample` : negative sample 개수
        - `iteration` : 학습 반복 횟수
        - `lr` : learning rate
        - `model` : `SkipGram` 클래스의 인스턴스
        - `optimizer` : `SGD` 클래스의 인스턴스
    - 메소드
        - `train()`
            - 입력 매개변수 
                - `train_dataloader`
            - Iteration 횟수만큼 input_file 학습 데이터를 학습한다. 매 epoch마다 for loop 돌면서 batch 단위 학습 데이터를 skip gram 모델에 학습함. 학습이 끝나면 word embedding을 output_file_name 파일에 저장.
- Reference
    - [Optimizer - SGD](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)

In [72]:
from bs4.builder import TreeBuilderRegistry
class Word2Vec:
    def __init__(self, 
                input_file: List[str],
                output_file_name: str,
                 device: str,
                 emb_dimension=300,
                 batch_size = 64,
                 window_size=5,
                 n_neg_sample = 5,
                 iteration=1,
                 lr = 0.02,
                 min_count=5):
        self.docs = input_file
        self.output_file_name = output_file_name
        self.word2count, self.word2id, self.id2word = make_vocab(input_file,min_count)
        self.device = device
        self.emb_size = len(self.word2id)
        self.emb_dimension = emb_dimension
        self.batch_size = batch_size
        self.window_size = window_size
        self.n_neg_sample = n_neg_sample
        self.iteration = iteration
        self.lr = lr
        self.model = SkipGram(vocab_size = len(self.word2id), emb_dimension = emb_dimension, device = device)
        self.optimizer = torch.optim.SGD(self.model.parameters(),lr=lr) # torch.optim.SGD 클래스 사용

        # train() 함수에서 만든 임베딩 결과 파일들을 저장할 폴더 생성 (os.makedirs 사용)
        os.makedirs('./'+output_file_name,exist_ok=True)
        
    
    def train(self, train_dataloader):
        
        # lr 값을 조절하는 스케줄러 인스턴스 변수를 생성
        self.scheduler = get_linear_schedule_with_warmup(
            optimizer = self.optimizer,
            num_warmup_steps=0,
            num_training_steps=self.iteration*len(train_dataloader)
        )
        
        for epoch in range(self.iteration):
            
            print(f"*****Epoch {epoch} Train Start*****")
            print(f"*****Epoch {epoch} Total Step {len(train_dataloader)}*****")
            total_loss, batch_loss, batch_step = 0,0,0

            for step, batch in enumerate(train_dataloader):
                batch_step+=1
                pos_u, pos_v = batch
                # negative data 생성
                neg_v = get_neg_v_negative_sampling(pos_u.shape[0], self.n_neg_sample)
                
                # 데이터를 tensor화 & device 설정
                pos_u = Variable(torch.LongTensor(pos_u)).to(device)
                pos_v = Variable(torch.LongTensor(pos_v)).to(device)
                neg_v = Variable(torch.LongTensor(neg_v)).to(device)

                # model의 gradient 초기화
                self.model.init_embedding()
                # optimizer의 gradient 초기화
                self.optimizer.zero_grad()

                # forward
                loss = self.model.forward(pos_u,pos_v,neg_v)

                # loss 계산
                loss.backward()
                # optimizer 업데이트
                self.optimizer.step()
                # scheduler 업데이트
                self.scheduler.step()

                batch_loss += loss.item()
                total_loss += loss.item()
                
                if (step%500 == 0) and (step!=0):
                    print(f"Step: {step} Loss: {batch_loss/batch_step:.4f} lr: {self.optimizer.param_groups[0]['lr']:.4f}")
                    # 변수 초기화    
                    batch_loss, batch_step = 0,0
            
            print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
            print(f"*****Epoch {epoch} Train Finished*****\n")
            
            print(f"*****Epoch {epoch} Saving Embedding...*****")
            self.model.save_embedding(self.id2word, os.path.join(self.output_file_name, f'w2v_{epoch}.txt'), True if 'cuda' in self.device.type else False)
            print(f"*****Epoch {epoch} Embedding Saved at {os.path.join(self.output_file_name, f'w2v_{epoch}.txt')}*****\n")

In [73]:
output_file = os.path.join(".", "word2vec_wiki")
# Word2Vec 클래스의 인스턴스 생성
w2v = Word2Vec(docs, output_file, device, n_neg_sample=10, iteration=3)

100%|██████████| 500/500 [00:00<00:00, 6916.32it/s]


In [74]:
# 학습 데이터 셋 및 데이터 로더 생성 (위에서 생성한 w2v의 attribute들을 argument에 적절히 넣기)
dataset = CustomDataset(docs, w2v.word2id, w2v.window_size)
train_dataloader = DataLoader(dataset,w2v.batch_size,shuffle=True)
len(train_dataloader)

16013

In [75]:
# 학습
w2v.train(train_dataloader)

*****Epoch 0 Train Start*****
*****Epoch 0 Total Step 16013*****
Step: 500 Loss: 487.9756 lr: 0.0198
Step: 1000 Loss: 487.9756 lr: 0.0196
Step: 1500 Loss: 487.9756 lr: 0.0194
Step: 2000 Loss: 487.9756 lr: 0.0192
Step: 2500 Loss: 487.9756 lr: 0.0190
Step: 3000 Loss: 487.9756 lr: 0.0188
Step: 3500 Loss: 487.9756 lr: 0.0185
Step: 4000 Loss: 487.9756 lr: 0.0183
Step: 4500 Loss: 487.9756 lr: 0.0181
Step: 5000 Loss: 487.9756 lr: 0.0179
Step: 5500 Loss: 487.9756 lr: 0.0177
Step: 6000 Loss: 487.9756 lr: 0.0175
Step: 6500 Loss: 487.9756 lr: 0.0173
Step: 7000 Loss: 487.9756 lr: 0.0171
Step: 7500 Loss: 487.9756 lr: 0.0169
Step: 8000 Loss: 487.9756 lr: 0.0167
Step: 8500 Loss: 487.9756 lr: 0.0165
Step: 9000 Loss: 487.9756 lr: 0.0163
Step: 9500 Loss: 487.9756 lr: 0.0160
Step: 10000 Loss: 487.9756 lr: 0.0158
Step: 10500 Loss: 487.9756 lr: 0.0156
Step: 11000 Loss: 487.9756 lr: 0.0154
Step: 11500 Loss: 487.9756 lr: 0.0152
Step: 12000 Loss: 487.9756 lr: 0.0150
Step: 12500 Loss: 487.9756 lr: 0.0148
Step:

In [58]:
next(w2v.model.parameters())

Parameter containing:
tensor([[ 1.4579e-03,  1.1121e-04, -1.1798e-03,  ..., -4.9304e-04,
         -1.1774e-03,  1.6205e-03],
        [ 6.6293e-04,  6.0215e-04,  9.7566e-04,  ..., -2.8999e-04,
          1.2292e-03,  1.2270e-03],
        [-1.4698e-03,  6.5783e-04, -1.8426e-04,  ...,  9.6384e-04,
          7.0798e-04, -2.2876e-05],
        ...,
        [-1.4436e-03, -5.6932e-05,  1.3793e-03,  ..., -1.3117e-03,
         -1.0156e-03,  1.5555e-03],
        [ 1.3746e-03, -1.2367e-03, -1.7306e-04,  ...,  4.5078e-04,
         -6.9273e-04,  7.2650e-04],
        [-1.1881e-03,  1.6646e-04, -2.1749e-04,  ..., -1.2870e-03,
          1.5915e-03, -1.1029e-03]], device='cuda:0', requires_grad=True)

### 유사한 단어 확인
- 사전에 존재하는 단어들과 유사한 단어를 검색해보자. Gensim 패키지는 유사 단어 외에도 단어간의 유사도를 계산하는 여러 함수를 제공한다. 실험을 통해 word2vec의 한계점을 발견했다면 아래에 markdown으로 작성해보자. 
- [Gensim 패키지 document](https://radimrehurek.com/gensim/models/keyedvectors.html)

In [51]:
import gensim

In [52]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/NLP/word2vec_wiki/w2v_2.txt', binary=False)

In [ ]:
word_vectors.most_similar(positive=None)

word2vec의 한계점은?